In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta
import utils
import scipy.signal as signal
import cf_xarray as cfxr
from datetime import timedelta


dask.config.set(**{'array.slicing.split_large_chunks': True})


## Import data and calculate GF

In [2]:
model_run_4x_dict = utils.model_run_4x_dict
model_run_control_dict = utils.model_run_picontrol_dict
#no NBP for GFDL

In [3]:
fg_co2_4x = {}
fg_co2_pictrl = {}
nbp_4x = {}
nbp_pictrl = {}
co2_4x = {}
co2_pictrl = {}
areacello = {}
areacella = {}

In [4]:
for m in model_run_4x_dict.keys():
    print(m)
    print('load 4x run')
    print('fg')
    fg_co2_4x[m] = xr.open_mfdataset(f'cmip6_data/fgco2_Omon_{model_run_4x_dict[m]}', use_cftime=True) #kgC/m2/s
    print('nbp')
    if m == 'UKESM1_r1' or m == 'MIROC' or m == 'NORESM2' or m == 'ACCESS' or m == 'CANESM5_r1p2' or m == 'CANESM5_r1p1':
        nbp_4x[m] = xr.open_mfdataset(f'cmip6_data/nbp_Lmon_{model_run_4x_dict[m]}', use_cftime=True) #kgC/m2/s
    if m == 'UKESM1_r1':
        co2_4x[m] = xr.open_mfdataset(f'cmip6_data/co2mass_Amon_{model_run_4x_dict[m]}', use_cftime=True) #kg
    
    

UKESM1_r1
load 4x run
fg
nbp
MIROC
load 4x run
fg
nbp
NORESM2
load 4x run
fg
nbp
ACCESS
load 4x run
fg
nbp
CANESM5_r1p2
load 4x run
fg
nbp
CANESM5_r1p1
load 4x run
fg
nbp


In [5]:
for m in model_run_control_dict.keys():
    print(m)
    print('load pi control run')
    fg_co2_pictrl[m] = xr.open_mfdataset(f'cmip6_data/fgco2_Omon_{model_run_control_dict[m]}', use_cftime=True, engine = 'netcdf4') #kg/m2/s 
    
    nbp_pictrl[m] = xr.open_mfdataset(f'cmip6_data/nbp_Lmon_{model_run_control_dict[m]}',use_cftime=True, engine = 'netcdf4') #kgC/m2/s 

    if m == 'UKESM1_r1' or m == 'NORESM2' or m == 'GFDL':
        co2_pictrl[m] = xr.open_mfdataset(f'cmip6_data/co2mass_Amon_{model_run_control_dict[m]}',use_cftime=True, engine = 'netcdf4') #kg
    
    areacello[m] = xr.open_mfdataset(f'cmip6_data/areacello_Ofx_{model_run_control_dict[m]}', use_cftime=True, engine = 'netcdf4')
    areacella[m] = xr.open_mfdataset(f'cmip6_data/areacella_fx_{model_run_control_dict[m]}', use_cftime=True, engine = 'netcdf4')


UKESM1_r1
load pi control run
MIROC
load pi control run
NORESM2
load pi control run
ACCESS
load pi control run
GFDL
load pi control run
CANESM5_r1p2
load pi control run
CANESM5_r1p1
load pi control run


## Find 3 times the initial co2 concentration


In [6]:
kgCO2_to_kgC = 1/3.67
vmr_conversion = 1.5172413793 ##see attrs for NORESM2, co2 is in vmr, not converted properly
t = 1 #years
three_ci = co2_pictrl['NORESM2']['co2mass'].mean(dim = 'time').values*vmr_conversion*3/(t)
three_ci*= kgCO2_to_kgC
three_ci

1802825042577700.5

In [7]:
#convert kg to kg/year
for m in ['UKESM1_r1']:
    seconds_per_yr = 60*60*24*365
    co2_4x[m]['G_atm'] = co2_4x[m]['co2mass'].diff('time')/(co2_4x[m]['time'].diff('time').astype('float64')/(1e9*seconds_per_yr)) #convert from ns to year via 1e9ns/s x 60s/min x 60min/hr x 24hr/day x 365day/yr
    co2_4x[m]['G_atm'] *= kgCO2_to_kgC
    co2_4x[m]['G_atm'].attrs = {'units':'kgC'}
    
for m in ['UKESM1_r1']:
    seconds_per_yr = 60*60*24*365
    co2_pictrl[m]['G_atm'] = co2_pictrl[m]['co2mass'].diff('time')/(co2_pictrl[m]['time'].diff('time').astype('float64')/(1e9*seconds_per_yr)) #convert from ns to year via 1e9ns/s x 60s/min x 60min/hr x 24hr/day x 365day/yr
    co2_pictrl[m]['G_atm'] *= kgCO2_to_kgC
    co2_pictrl[m]['G_atm'].attrs = {'units':'kgC'}

In [8]:
## fix the times so that they line up according to the notes above
m = 'NORESM2'
nbp_pictrl[m]['time'] = nbp_pictrl[m]['time'] -timedelta(365*1599)
fg_co2_pictrl[m]['time'] = fg_co2_pictrl[m]['time'] - timedelta(365*1599)


m = 'UKESM1_r1'
nbp_pictrl[m]['time'] = nbp_pictrl['UKESM1_r1']['time'] - timedelta(360*110)
fg_co2_pictrl[m]['time'] = fg_co2_pictrl['UKESM1_r1']['time'] - timedelta(360*110)
co2_pictrl[m]['time'] = co2_pictrl['UKESM1_r1']['time'] - timedelta(360*110)

m = 'CANESM5_r1p2'
nbp_pictrl[m]['time'] = nbp_pictrl['CANESM5_r1p2']['time']- timedelta(365*3700)
fg_co2_pictrl[m]['time'] = fg_co2_pictrl['CANESM5_r1p2']['time']- timedelta(365*3700)

m = 'CANESM5_r1p1'
nbp_pictrl[m]['time'] = nbp_pictrl['CANESM5_r1p1']['time']- timedelta(365*3351)
fg_co2_pictrl[m]['time'] = fg_co2_pictrl['CANESM5_r1p1']['time']- timedelta(365*3351)


In [9]:
#define our output grid size

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(0, 360, 1)),
        "lat_b": (["lat_b"], np.arange(-90.,91.,1.0)),
        "lon_b":(["lon_b"], np.arange(.5, 361.5, 1.0))
    }
)
A = utils.find_area(ds_out, R = 6.3781e6)

#replace GFDL areacello with this-- areacello doesn't match up

areacello['GFDL'] = A

In [10]:
fg_co2 = {}
nbp = {}

for m1 in model_run_4x_dict.keys():
    m2 = m1
    fg_co2[m1] = fg_co2_4x[m1] - fg_co2_pictrl[m2]
    if m1 != 'GFDL':
        nbp[m1] = nbp_4x[m1] - nbp_pictrl[m2]
    
    if m2 == 'GFDL':
        fg_co2[m1]['fgco2'] = fg_co2[m1]['fgco2']*seconds_per_yr*areacello[m2]
    else:
        fg_co2[m1]['fgco2'] = fg_co2[m1]['fgco2']*seconds_per_yr*areacello[m2]['areacello']
    if m1 != 'GFDL':
        nbp[m1]['nbp'] = nbp[m1]['nbp']*seconds_per_yr*areacella[m2]['areacella']

In [11]:
co2_dif = {}
for m1 in co2_4x.keys():
    m2 = m1
    print(m1, m2)
    co2_dif[m1] = co2_4x[m1]['G_atm'] - co2_pictrl[m2]['G_atm'] #kg/yr

UKESM1_r1 UKESM1_r1


In [12]:
#total sinks
emis_sinks = {}
for m in nbp.keys():
    if 'lat' in list(fg_co2[m]['fgco2'].dims):
        emis_sinks[m] = nbp[m]['nbp'].sum(dim = ['lat','lon']) + fg_co2[m]['fgco2'].sum(dim = ['lat','lon'])
    elif 'i' in list(fg_co2[m]['fgco2'].dims):
        emis_sinks[m] = nbp[m]['nbp'].sum(dim = ['lat','lon']) + fg_co2[m]['fgco2'].sum(dim = ['i','j'])
    elif 'x' in list(fg_co2[m]['fgco2'].dims):
        emis_sinks[m] = nbp[m]['nbp'].sum(dim = ['lat','lon']) + fg_co2[m]['fgco2'].sum(dim = ['x','y'])

In [13]:
#fix the times to all be the same, weight by month and take the mean

for m in nbp.keys():    
    times = emis_sinks[m].time.get_index('time')
    weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
    weights = xr.DataArray(weights, [('time', emis_sinks[m]['time'].values)]).astype('float')
    emis_sinks[m] =  (emis_sinks[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

In [14]:
G_atm = {}
for m in co2_dif.keys():    
    times = co2_dif[m].time.get_index('time')
    weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
    weights = xr.DataArray(weights, [('time', co2_dif[m]['time'].values)]).astype('float')
    G_atm[m] =  (co2_dif[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

## Add in the pulse, which is not accounted for bc it is infinite 
We discretize it and make it as 3ci where ci is the initial pictrl concentration

In [20]:
G_atm['UKESM1_r1'].loc[dict(year = 1850)] = three_ci

In [21]:
co2_pictrl['NORESM2']['co2mass'].mean(dim = 'time').values

array(1.453596e+15, dtype=float32)

In [22]:
for m in emis_sinks.keys():
    emis_sinks[m]['year'] = np.arange(0, len(emis_sinks[m]['year']))

In [23]:
for m in G_atm.keys():
    G_atm[m]['year'] = np.arange(0, len(G_atm[m]['year']))

In [24]:
def diff_lists(list1, list2):
    return list(set(list1).symmetric_difference(set(list2)))  # or return list(set(list1) ^ set(list2))

emis_co2 = {}
for m in G_atm.keys():
        emis_co2[m] = G_atm[m] + emis_sinks[m]

for m in diff_lists(emis_sinks.keys(), G_atm.keys()):
        emis_co2[m] = G_atm['UKESM1_r1'] + emis_sinks[m] #use UKESM1_r1 co2 mass bc these runs don't have their own
    

In [25]:
emis_co2_GtC = {}
kg_to_Gt = 1e-12

for m in emis_co2.keys():
    emis_co2_GtC[m] = emis_co2[m]*kg_to_Gt
    

In [26]:
emis_co2_GtC_ds = xr.concat([emis_co2_GtC[m] for m in emis_co2_GtC.keys()], pd.Index([m for m in emis_co2_GtC.keys()], name='model'), coords='minimal')

## Save out

In [27]:
emis_co2_GtC_ds.to_netcdf('Outputs/4x_emis_profile_full.nc4')


In [ ]:
emis_co2_GtC_ds